In [ ]:
# 필요한 라이브러리 로드
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings

warnings.filterwarnings('ignore')

In [ ]:
# 환율 데이터 로드
exchange = pd.read_csv('../data/USD_KRW.csv')

In [ ]:
# 5일 이동평균선 생성
MA5 = exchange['close'].rolling(window=5).mean()

In [6]:
# 인덱스 날짜 컬럼으로 변경
exchange.index = exchange['date'].map(lambda x : pd.to_datetime(x))

In [ ]:
# 이미지 저장할 폴더 생성
os.makedirs('../image')

In [ ]:
# 이미지 변환 함수 선언
    # n : n일치 그래프 생성
    # cate : 환율/환율 이평선
    # witdh : 이미지 가로 길이
    # height : 이미지 세로 길이 
def image_rate(n, cate, witdh, height) :
  for indx in range(len(exchange) - n) :

    df = exchange.iloc[indx : indx + n, : ]

    fig, ax = plt.subplots(figsize=(witdh, height))
    fig.patch.set_facecolor('black')
    ax.set_facecolor('black')

    for i, (idx, row) in enumerate(df.iterrows()):
      x = i  # X축 위치
      # 수직선: High to Low
      ax.plot([x, x], [row['Low'], row['High']], color='white', linewidth=1)
      # 왼쪽 수평선: Open
      ax.plot([x - 0.5, x], [row['Open'], row['Open']], color='white', linewidth=1)
      # 오른쪽 수평선: Close
      ax.plot([x, x + 0.5], [row['Close'], row['Close']], color='white', linewidth=1)
      if cate.find('이평선') != -1 :
        # 5일 이동평균선
        ax.plot(range(len(df)), MA5[indx:indx+n], color='white', linewidth=0.5)

    # 시각 설정
    ax.grid(False)
    ax.set_facecolor('white')

    # 4. 여백 및 축 제거
    ax.set_xlim(-0.5, len(df)+0.5)
    if cate.find('이평선') == -1 :
      ax.set_ylim(df['Low'].min(), df['High'].max())
    else :
      ax.set_ylim(min(MA5[indx:indx+n].min(), df['Low'].min()), max(MA5[indx:indx+n].max(), df['High'].max()))
    ax.axis('off')
    fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

    fig.savefig(f'../image/{n}일 {cate}/{exchange.iloc[indx, : ]["date"]}.png', dpi=100, bbox_inches='tight', pad_inches=0)

    plt.close(fig)

In [ ]:
image_rate(5, '환율', 15/100, 32/100)
image_rate(5, '환율 이평선', 15/100, 32/100)
image_rate(20, '환율', 60/100, 64/100)
image_rate(20, '환율 이평선', 60/100, 64/100)
image_rate(60, '환율', 180/100, 96/100)
image_rate(60, '환율 이평선', 180/100, 96/100)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import Counter
import seaborn as sns
import os
from PIL import Image
from sklearn.metrics import classification_report, roc_curve, auc

# 참고문헌대로 만든 모델

# 5일치 데이터용
def day5_model():
  model = models.Sequential()

  # 첫 번째 Convolutional 레이어
  model.add(layers.Conv2D(
    filters=64,             # 필터 개수
    kernel_size=(5, 3),     # 커널 크기
    padding='same',         # 출력 크기를 입력과 동일하게 유지
    input_shape=(32,15,1)
  ))
  model.add(layers.LeakyReLU())           # LeakyReLU 활성화 함수
  model.add(layers.MaxPooling2D(pool_size=(2, 1)))  # 2x1 Max Pooling

  # 두 번째 Convolutional 레이어
  model.add(layers.Conv2D(
    filters=128,
    kernel_size=(5, 3),
    padding='same'
  ))
  model.add(layers.LeakyReLU())
  model.add(layers.MaxPooling2D(pool_size=(2, 1)))

  # Feature map을 1차원 벡터로 펼침 (Flatten)
  model.add(layers.Flatten())

  # Fully Connected (Dense) Layer
  model.add(layers.Dense(15360))

  # 출력층
  model.add(layers.Dense(1, activation='softmax'))

  # 모델 컴파일
  model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
  )

  return model

# 20일치 데이터용
def day20_model():
  model = models.Sequential()

  # 첫 번째 Convolutional 레이어
  model.add(layers.Conv2D(
    filters=64,             # 필터 개수
    kernel_size=(5, 3),     # 커널 크기
    padding='same',         # 출력 크기를 입력과 동일하게 유지
    input_shape=(64,60,1)
  ))
  model.add(layers.LeakyReLU())           # LeakyReLU 활성화 함수
  model.add(layers.MaxPooling2D(pool_size=(2, 1)))  # 2x1 Max Pooling

  # 두 번째 Convolutional 레이어
  model.add(layers.Conv2D(
    filters=128,
    kernel_size=(5, 3),
    padding='same'
  ))
  model.add(layers.LeakyReLU())
  model.add(layers.MaxPooling2D(pool_size=(2, 1)))

  # 세 번째 Convolutional 레이어
  model.add(layers.Conv2D(
    filters=256,
    kernel_size=(5, 3),
    padding='same'
  ))
  model.add(layers.LeakyReLU())
  model.add(layers.MaxPooling2D(pool_size=(2, 1)))

  # Feature map을 1차원 벡터로 펼침 (Flatten)
  model.add(layers.Flatten())

  # Fully Connected (Dense) Layer
  model.add(layers.Dense(46080))

  # Softmax 출력층 (클래스 수만큼 출력)
  model.add(layers.Dense(1, activation='softmax'))

  # 모델 컴파일
  model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
  )
  return model

# 60일치 데이터용
def day60_model():
  model = models.Sequential()

  # 첫 번째 Convolutional 레이어
  model.add(layers.Conv2D(
    filters=64,             # 필터 개수
    kernel_size=(5, 3),     # 커널 크기
    padding='same',         # 출력 크기를 입력과 동일하게 유지
    input_shape=(96,180,1)
  ))
  model.add(layers.LeakyReLU())           # LeakyReLU 활성화 함수
  model.add(layers.MaxPooling2D(pool_size=(2, 1)))  # 2x1 Max Pooling

  # 두 번째 Convolutional 레이어
  model.add(layers.Conv2D(
    filters=128,
    kernel_size=(5, 3),
    padding='same'
  ))
  model.add(layers.LeakyReLU())
  model.add(layers.MaxPooling2D(pool_size=(2, 1)))

  # 세 번째 Convolutional 레이어
  model.add(layers.Conv2D(
    filters=256,
    kernel_size=(5, 3),
    padding='same'
  ))
  model.add(layers.LeakyReLU())
  model.add(layers.MaxPooling2D(pool_size=(2, 1)))

  # 네 번째 Convolutional 레이어
  model.add(layers.Conv2D(
    filters=512,
    kernel_size=(5, 3),
    padding='same'
  ))
  model.add(layers.LeakyReLU())
  model.add(layers.MaxPooling2D(pool_size=(2, 1)))

  # Feature map을 1차원 벡터로 펼침 (Flatten)
  model.add(layers.Flatten())

  # Fully Connected (Dense) Layer
  model.add(layers.Dense(184320))

  # Softmax 출력층 (클래스 수만큼 출력)
  model.add(layers.Dense(1, activation='softmax'))

  # 모델 컴파일
  model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
  )

  return model

# 경량화 모델
def simple_model(input_shape) :

  model = tf.keras.Sequential([
    # Input Layer
    tf.keras.layers.Input(shape=input_shape),

    # First Convolutional Block
    tf.keras.layers.Conv2D(32, (5, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 1)),
    tf.keras.layers.Dropout(0.25),

    # Second Convolutional Block
    tf.keras.layers.Conv2D(16, (5, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 1)),
    tf.keras.layers.Dropout(0.25),

    # Flatten and Dense Layers
    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.25),

    # Output Layer
    tf.keras.layers.Dense(1, activation = 'sigmoid')
  ])

  # 모델 컴파일
  model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
  )

  return model

# x, y 스플릿 함수
def seq_split(x, y, per=0.7) :
  x_train = x[ : int(len(y)*per)]
  x_test = x[int(len(y)*per) : len(y)]
  y_train = y[ : int(len(y)*per)]
  y_test = y[int(len(y)*per) : ]
  return x_train, x_test, y_train, y_test

#  평가용 함수
def testing(y_test, y_pred) :
  print(classification_report(y_test, y_pred, target_names=["환율 미상승", "환율 상승"]))

  # AUC 커브
  fpr, tpr, thresholds = roc_curve(y_test, y_pred)
  roc_auc = auc(fpr, tpr)

  plt.figure(figsize=(8, 6))
  plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
  plt.plot([0, 1], [0, 1], color='gray', linestyle='--')  # 기준선
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('False Positive Rate (FPR)')
  plt.ylabel('True Positive Rate (TPR)')
  plt.title('Receiver Operating Characteristic (ROC) Curve')
  plt.legend(loc='lower right')
  plt.grid(True)
  plt.show()

# 독립변수 로드 함수
def load_x(path) :
  image_array = []
  datas = os.listdir(path)
  datas.sort()
  for data in datas:
      img = Image.open(path+data).convert('L')
      img = np.array(img) / 255.0  # 정규화
      img = np.expand_dims(img, axis=-1)
      image_array.append(img)
  return np.array(image_array)

# y값 설정 함수
# n일 환율 데이터 사용, 기준일로부터 m일 후 환율종가 l% 상승 여부(ㅣ% 상승 : 1, 상승하지 않음 : 0)
def create_y(df, n, m, l) :
  y = []
  for i in range(len(df)-n-m) :
    if ((df.iloc[i+n-1+m,4]/df.iloc[i+n-1,4]) - 1) >= (l/100) :
      y.append(1)
    else :
      y.append(0)
  return np.array(y)